# Stabler's tree-based formalism

## The basic feature calculus

In [ ]:
type Phon = String

data CatSel = N | V | D deriving Eq

instance Show CatSel where
    show N = "n"
    show V = "v"
    show D = "d"

data CatLic = Wh | Top deriving Eq

instance Show CatLic where
    show Wh = "wh"
    show Top = "top"

data Feature = Sel { selector :: Bool, cat :: CatSel } | Lic { goal :: Bool, catLic :: CatLic } deriving Eq

instance Show Feature where
    show (Sel True c) = "=" ++ show c
    show (Sel False c) = show c
    show (Lic True c) = "-" ++ show c
    show (Lic False c) = "+" ++ show c
    
xor :: Bool -> Bool -> Bool
t `xor` t' = t || t' && not (t && t')
    
cancel :: Feature -> Feature -> Either String Feature
cancel (Sel t c) (Sel t' c') = c == c' && t `xor` t'

## A datatype for lexical items and MGs

In [ ]:
data Lex = Lex { phon :: Phon, f :: [Feature] } deriving Eq

type MG = [Lex]

instance Show Lex where
    show (Lex phon xs) = phon ++ ":: " ++ show xs
        
the :: Lex
the = Lex { phon = "the", f = [Sel True N, Sel False D]}
dog :: Lex
dog = Lex { phon = "dog", f = [Sel False N]}

the
dog

## A datatype for derived representations

Lexical items label tree nodes (bare phrase structure).

In [ ]:
import Diagrams.TwoD.Layout.Tree

type SynTree = BTree Lex

tMerge :: SynTree -> SynTree -> Lex -> Lex -> Lex -> SynTree

# Displaying binary trees in haskell via `diagrams`

In [ ]:
{-# LANGUAGE NoMonomorphismRestriction,FlexibleContexts, LambdaCase #-}

import Diagrams.Prelude
import Diagrams.Backend.Cairo.CmdLine (B)
import Data.Maybe (fromJust)

t = BNode ">" (BNode ">" (leaf "<") (leaf ">")) (BNode "<" (leaf ">") (leaf "<"))

In [ ]:
drawNode n = text n # lw 1 # fontSizeL 0.2 # fc black # scaleX 1
 <> rect 0.5 0.5 # fc white

drawBranch = (~~)

drawLex :: Lex -> QDiagram B V2 Double Any
drawLex lex = text (show lex) # lw 1 # fontSizeL 0.05 # fc black # scaleX 1
 <> rect 0.5 0.5 # fc white

myRenderTree = renderTree drawNode drawBranch . fromJust . uniqueXLayout 1 1

myRenderSynTree = renderTree drawLex drawBranch . fromJust . uniqueXLayout 1 1

In [ ]:
myRenderTree t :: QDiagram B V2 Double Any

In [ ]:
myRenderSynTree $ fromJust exTree2

### An aside: conflated MGs

In [ ]:
data Cat = N | V | D deriving Eq
data FType = Selector | Selectee | Licensor | Licensee deriving Eq

data Feature = F {cat :: Cat,f :: FType} deriving Eq